In [169]:
import pprint, json, math, os, sys, camelot
import fitz
import pandas as pd
import numpy as np
from collections import defaultdict

dir_path = "C:\\Users\\Kaustubh.keny\\OneDrive - Cogencis Information Services Ltd\\Documents\\mywork-repo\\"
fund_path = "C:\\Users\\Kaustubh.keny\\OneDrive - Cogencis Information Services Ltd\\Documents\\Dec 24\\"

# dir_path = "C:\\Users\\rando\\OneDrive\\Documents\\mywork-repo"
# fund_path =  "C:\\Users\\rando\\OneDrive\\Documents\\Dec 24"
sys.path.append(os.path.abspath(dir_path))

from app.utils import Helper
from app.fund_regex import *

dry_path = r'\data\output\DryRun.pdf'
fin_path = r'\data\input\financial_indices.xlsx'
mutual_fund = Helper.get_fund_paths(fund_path)

In [3]:
def extract_clipped_data(input:str, pages:list, bboxes:list):
        
        document = fitz.open(input)
        final_list = []
    
        for pgn in pages:
            page = document[pgn]
            
            all_blocks = [] #store every data from bboxes
            
            for bbox in bboxes:
                blocks, seen_blocks = [], set()  #store unique blocks based on content and bbox
                
                page_blocks = page.get_text('dict', clip=bbox)['blocks']
                for block in page_blocks:
                    if block['type'] == 0 and 'lines' in block: #type 0 means text block
                        #hash_key
                        block_key = (tuple(block['bbox']), tuple(tuple(line['spans'][0]['text'] for line in block['lines'])))
                        if block_key not in seen_blocks:
                            seen_blocks.add(block_key)
                            blocks.append(block)

                sorted_blocks = sorted(blocks, key=lambda x: (x['bbox'][1], x['bbox'][0]))
                all_blocks.append(sorted_blocks)

            final_list.append({
                "pgn": pgn,
                "block": all_blocks #will be list[list,list,..]
            })

        document.close()
        return final_list

In [4]:
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        sorted_blocks = sorted(blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in sorted_blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

      
        final_list.append({
            "pgn": pgn,
            "blocks": extracted_blocks
        })

    doc.close()

    return final_list

def extract_spans(data):
    final_span = []
    for pgn,blocks in enumerate(data):
        spans = []
        print(f"___{pgn}___")
        for num,block in enumerate(blocks.get('blocks',[])):
            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    if num in range(3,10):
                        print(span['text'], span['size'])
        final_span.append(spans)
    return final_span

def get_proper_fund_names(path:str,pages:list):
        
    doc = fitz.open(path)
    final_fund_names = dict()
    
    for pgn in range(doc.page_count):
        text_all = ''
        if pgn in pages:
            # print(pgn)
            page = doc[pgn]            
            blocks = page.get_text("dict")['blocks']
            
            sorted_blocks = sorted(blocks,key=lambda k:(k['bbox'][1],k['bbox'][0]))
            for count,block in enumerate(sorted_blocks):
                for line in block.get("lines", []):
                    for span in line.get("spans", []):
                        text = span['text'].strip()
                        if count in range(0,1):
                            text_all+=f" {text}"
            print(text_all)
        pattern = r"MIRAE ASSET.*?\b(?:ETF|EOF|FOF|FTF|FUND)\b"
        if matches := re.findall(pattern, text_all.strip(), re.DOTALL):
            final_fund_names[pgn] = matches[0]
        else:
            final_fund_names[pgn] = ""
    return final_fund_names
  
def get_clipped_data(input:str, bboxes:list[set]):
    
        document = fitz.open(input)
        final_list = []
        
        for pgn in range(document.page_count):
            page = document[pgn]

            blocks = []
            for bbox in bboxes:
                blocks.extend(page.get_text('dict', clip = bbox)['blocks']) #get all blocks
            
            filtered_blocks = [block for block in blocks if block['type']== 0 and 'lines' in block]
            # sorted_blocks = sorted(filtered_blocks, key= lambda x: (x['bbox'][1], x['bbox'][0]))
            
            final_list.append({
            "pgn": pgn,
            "block": filtered_blocks
            })
            
            
        document.close()
        return final_list
    
def get_clipped_text(input:str, bboxes:list[set]):

    document = fitz.open(input)
    final_list = []
    
    for pgn in range(document.page_count):
        page = document[pgn]
        blocks = []
        for bbox in bboxes:
            blocks = page.get_text('text', clip = bbox).split('\n') #get all blocks
  
        final_list.append({
        "pgn": pgn,
        "block": blocks
        })   
    document.close()
    return final_list

In [40]:
sample_path  = mutual_fund['Nippon India Mutual Fund']
pages = [12, 14,16]
bboxes = [(200, 50, 380, 812),(380, 50, 580, 812)]

In [ ]:
lines = [
    ((220, 0), (220, 812)),# Vertical line
    #((420, 0), (420, 1000))
]

rectangles = [(0, 50, 200, 812),(200, 50, 380, 812),(380, 50, 580, 812)]
pages = [i for i in range(1,110)]
Helper.draw_lines_on_pdf(sample_path, lines, rectangles, pages, dir_path +dry_path)


Modified PDF saved to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\data\output\DryRun.pdf


In [3]:
data = Helper.get_all_pdf_data(sample_path)

In [25]:
def get_someting(path:str):
    pattern = r"\bFUNDS AT A GLANCE\b"
    
    with fitz.open(path) as doc:
        for pgn,page in enumerate(doc):
            page_text ="".join(page.get_text().split("\n"))
            # print(page_text)
            if re.match(pattern,page_text):
                print(pgn)

In [42]:
sample_path

'C:\\Users\\Kaustubh.keny\\OneDrive - Cogencis Information Services Ltd\\Documents\\Dec 24\\Nippon India Mutual Fund\\33_31-Dec-24_FS.pdf'

In [211]:
def via_block(path:str):
    
    pattern = r"FUNDS AT A GLANCE"
    amc_pattern = "^(Nippon India|CPSE).*(?=Plan|Sensex|Fund|Path|ETF|FOF|EOF|Funds|$)"
    imp_pages = []
    with fitz.open(path) as doc:
        for pgn, page in enumerate(doc):
                page_blocks = page.get_text("dict")["blocks"]
                sorted_blocks = sorted(page_blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
                for block_count, block in enumerate(sorted_blocks[:10]):
                    if "lines" not in block:
                        continue
                    for line in block["lines"]:
                        for span in line["spans"]:
                            text = span["text"].strip()
                            if re.match(pattern,text):
                                imp_pages.append(pgn)
                                
        amc_fund = defaultdict(list)
    
        for pgn in imp_pages:
            page = doc[pgn]
            page_blocks = page.get_text("dict")["blocks"]
            sorted_blocks = sorted(page_blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
            for block_count, block in enumerate(sorted_blocks):
                if "lines" not in block:
                    continue
                for line in block["lines"]:
                    for span in line["spans"]:
                        text = span["text"].strip()
                        if re.match(amc_pattern,text):
                            amc_fund[pgn].append(text)
                            
    return imp_pages, dict(amc_fund)          

In [214]:
pages,amc = via_block(sample_path)
pages = list(map(str,pages))

In [226]:
final_scheme = []
for key, value in amc.items():
    # print(key)
    set1 = ['Type of Scheme']+value[:4]
    set2 = ['Type of Scheme']+value[4:]
    final_scheme.append(set1)
    final_scheme.append(set2)

In [ ]:
final_scheme

In [183]:

imp_pages = ",".join(pages)
tables = camelot.read_pdf(sample_path,pages=imp_pages, flavor="lattice", line_scale = 40)  #table_areas = ["0,0,580,690"]            
# html_table = tables[0].df.to_html(index=False, header=False)

In [233]:
final_tables = []
for i,table in enumerate(tables):
    df = table.df
    if df.shape[1]<3:
        continue
    df= df.map(lambda x: " ".join(x.split("\n")).strip())
    df = df.map(lambda x: np.nan if not x.strip() else x)
    
    df.set_index(df.columns[0], inplace=True)
    
    check = "Scheme Name"
    if check in df.index:
        df.drop(check, inplace=True)
    
    check = "Market Capitalization"
    if check in df.index:
        df.drop(check, inplace=True)
        
    df_cleaned = df[~df.index.isna()]  # Remove NaN index
    df_cleaned = df_cleaned[df_cleaned.index != ""]  # Remove empty string index

    df_cleaned = df_cleaned.reset_index()
    df_fill = df_cleaned.ffill(axis=1)
    df_fill.to_excel(f"sample{i}.xlsx", index=False)
    final_tables.append(df_fill)
    # print(final_scheme[i-1])
    # print(i,df_fill.shape)

In [234]:
len(final_tables)

22

In [235]:
len(final_scheme)

24

In [163]:
data = tables[2].df.set_index(df.columns[0])
# if "Scheme Name" in data.index:
#     data.drop("Scheme Name", inplace=True)

In [164]:
data

,1,2,3,4
0,,,,
Scheme Name,Nippon India Focused Equity Fund,Nippon India Small Cap Fund,Nippon India Value Fund,Nippon India Multi Cap Fund
Type of Scheme,An open ended Multi Cap Equity Scheme investin...,An\nopen-ended\nequity\nscheme\npredominantly\...,An\nopen\nended\nequity\nscheme\nfollowing\na\...,Multi Cap\nFund\n- An\nopen\nended\nequity\nsc...
Allotment Date,"April 28, 2018 (Date of Recategorisation)","September 16, 2010","June 08, 2005","March 28, 2005"
Investment Objective,The primary investment objective of\nthe schem...,The primary investment objective of\nthe schem...,The primary investment objective of\nthis sche...,The primary investment objective of\nthe schem...
Fund Manager,Vinay Sharma#\nRishit Parikh (Assistant Fund M...,Samir Rachh#,"Dhrumil Shah,\nMeenakshi Dawar #",Sailesh Raj Bhan\nAshutosh Bhargava (Co-Fund M...
Benchmark,BSE 500 TRI,Nifty Smallcap 250 TRI,Nifty 500 TRI,NIFTY 500 Multicap 50:25:25 TRI
Entry Load,Not Applicable. The upfront commission on inve...,,,
Exit Load,10% of\nthe units allotted shall be redeemed w...,1% if\nredeemed\nor\nswitched\nout\non\nor\nbe...,10% of\nthe units allotted shall be redeemed w...,
Investment Plans/ Options,Under both Direct and other than Direct plan\n...,,,


In [148]:
indexes =["Scheme Name"] + tables[1].df.iloc[:,0].tolist()

In [149]:
indexes

['Scheme Name',
 'Type of Scheme',
 'Allotment Date',
 'Investment Objective',
 'Fund Manager',
 'Benchmark',
 'Entry Load',
 'Exit Load',
 'Investment Plans/ Options',
 'Minimum Application Amount',
 'Market Capitalization']